# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [13]:
import requests
import pyowm
import json

api_key = '9be55d70c16bbbd108056ecff0b2613f'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [14]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'q': 'London',
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 292.82,
  'feels_like': 291.86,
  'temp_min': 292.01,
  'temp_max': 293.45,
  'pressure': 1018,
  'humidity': 39,
  'sea_level': 1018,
  'grnd_level': 1014},
 'visibility': 10000,
 'wind': {'speed': 5.14, 'deg': 240},
 'clouds': {'all': 69},
 'dt': 1748009966,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1747972678,
  'sunset': 1748030212},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [15]:
from pyowm import OWM

# Initialize OWM with your API key
owm = OWM(api_key)

# Get weather manager (handles all weather-related requests)
mgr = owm.weather_manager()

# Now get current weather for London
observation = mgr.weather_at_place('London,GB')
weather = observation.weather

print(f"Weather in London: {weather.detailed_status}")
print(f"Temperature: {weather.temperature('celsius')['temp']} °C")
print(f"Wind speed: {weather.wind()['speed']} m/s")
print(f"Humidity: {weather.humidity} %")

Weather in London: broken clouds
Temperature: 19.42 °C
Wind speed: 5.14 m/s
Humidity: 40 %


## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [23]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}K, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 292.31K, Humidity: 40%, Wind Speed: 5.14 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [24]:
owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place('London,GB')
weather = observation.weather

temperature = weather.temperature('celsius')['temp']
humidity = weather.humidity
wind_speed = weather.wind()['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 19.42°C, Humidity: 40%, Wind Speed: 5.14 m/s


## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [25]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

Then, request the weather forecast for that latitude/longitude:

In [26]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-05-23 15:00:00, Temp: 292.46°C
Time: 2025-05-23 18:00:00, Temp: 291.31°C
Time: 2025-05-23 21:00:00, Temp: 288.07°C
Time: 2025-05-24 00:00:00, Temp: 285.35°C
Time: 2025-05-24 03:00:00, Temp: 284.17°C
Time: 2025-05-24 06:00:00, Temp: 284.93°C
Time: 2025-05-24 09:00:00, Temp: 288.66°C
Time: 2025-05-24 12:00:00, Temp: 294.49°C
Time: 2025-05-24 15:00:00, Temp: 295.26°C
Time: 2025-05-24 18:00:00, Temp: 292.83°C
Time: 2025-05-24 21:00:00, Temp: 289.2°C
Time: 2025-05-25 00:00:00, Temp: 287.82°C
Time: 2025-05-25 03:00:00, Temp: 287.55°C
Time: 2025-05-25 06:00:00, Temp: 287.29°C
Time: 2025-05-25 09:00:00, Temp: 288.48°C
Time: 2025-05-25 12:00:00, Temp: 291.89°C
Time: 2025-05-25 15:00:00, Temp: 290.6°C
Time: 2025-05-25 18:00:00, Temp: 289.56°C
Time: 2025-05-25 21:00:00, Temp: 285.95°C
Time: 2025-05-26 00:00:00, Temp: 284.28°C
Time: 2025-05-26 03:00:00, Temp: 283.18°C
Time: 2025-05-26 06:00:00, Temp: 284.62°C
Time: 2025-05-26 09:00:00, Temp: 289.05°C
Time: 2025-05-26 12:00:00, Temp: 290

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [27]:
owm = OWM(api_key)
mgr = owm.weather_manager()

# Get forecast for New York City (the method handles geocoding internally)
forecast = mgr.forecast_at_place("New York,US", "3h")

# The forecast object contains multiple weather entries every 3 hours
for weather in forecast.forecast:
    time = weather.reference_time('iso')  # ISO format datetime string
    temp_c = weather.temperature('celsius')['temp']
    print(f"Time: {time}, Temp: {temp_c}°C")

Time: 2025-05-23 15:00:00+00:00, Temp: 11.02°C
Time: 2025-05-23 18:00:00+00:00, Temp: 12.31°C
Time: 2025-05-23 21:00:00+00:00, Temp: 14.61°C
Time: 2025-05-24 00:00:00+00:00, Temp: 14.44°C
Time: 2025-05-24 03:00:00+00:00, Temp: 11.98°C
Time: 2025-05-24 06:00:00+00:00, Temp: 10.46°C
Time: 2025-05-24 09:00:00+00:00, Temp: 9.47°C
Time: 2025-05-24 12:00:00+00:00, Temp: 11.3°C
Time: 2025-05-24 15:00:00+00:00, Temp: 12.74°C
Time: 2025-05-24 18:00:00+00:00, Temp: 14.42°C
Time: 2025-05-24 21:00:00+00:00, Temp: 15.62°C
Time: 2025-05-25 00:00:00+00:00, Temp: 13.68°C
Time: 2025-05-25 03:00:00+00:00, Temp: 12.44°C
Time: 2025-05-25 06:00:00+00:00, Temp: 11.46°C
Time: 2025-05-25 09:00:00+00:00, Temp: 10.7°C
Time: 2025-05-25 12:00:00+00:00, Temp: 12.69°C
Time: 2025-05-25 15:00:00+00:00, Temp: 17.74°C
Time: 2025-05-25 18:00:00+00:00, Temp: 16.88°C
Time: 2025-05-25 21:00:00+00:00, Temp: 17.26°C
Time: 2025-05-26 00:00:00+00:00, Temp: 16.08°C
Time: 2025-05-26 03:00:00+00:00, Temp: 14.06°C
Time: 2025-05-26

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [32]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = ((temperature_celsius - 273.15) * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}K, Fahrenheit: {temperature_fahrenheit:.2f}°F")

Temperature in Celsius: 292.31K, Fahrenheit: 66.49°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [33]:
owm = OWM(api_key)
mgr = owm.weather_manager()

observation = mgr.weather_at_place('London,GB')
weather = observation.weather

# Get temperature in Celsius (pyowm returns a dict with min, max, temp)
temp_celsius = weather.temperature('celsius')['temp']

# Get temperature in Fahrenheit similarly
temp_fahrenheit = weather.temperature('fahrenheit')['temp']

print(f"Temperature in Celsius: {temp_celsius}°C, Fahrenheit: {temp_fahrenheit}°F")

Temperature in Celsius: 19.72°C, Fahrenheit: 67.5°F
